In [ ]:
import numpy as np 
import pandas as pd
import multiprocessing as mp
from multiprocessing import Pool
from numba import jit
import json

In [ ]:
df = pd.read_csv('../datalets/datalet2.csv',index_col=False)

In [ ]:
headers = []

for i in range(df.shape[1]):
    headers.append('period{}'.format(i))
 

df.columns = headers    

display(df)

In [ ]:
counts = {}
for x in range(len(df['period0'].value_counts())):
    counts['{}.0'.format(x)]= df['period0'].value_counts()[x]
    
print(counts)


In [ ]:
df['ydata'] = df['period1']
output = pd.pivot_table(data=df, index=['period0'], columns=['period1'], values='ydata', aggfunc='count')
display(output)



In [ ]:
#This is a the stochastic markov chain that tells you, if you start at a particular MEV state, where might you end up I am just transposing it for the calculation and it will be transposed back after

tmatrix = output.transpose(copy=False)

#This loop divides by the total counts in each row, to work out the conditional probability of going from one state to another. 

for x in range(len(counts)):
    tmatrix[x] = (tmatrix[x] / counts['{}.0'.format(x)])


transition_matrix = tmatrix.transpose(copy=False)

display(transition_matrix)